**Import statements**

In [63]:
from numpy.random import seed
seed(888) #random number
import tensorflow as tf
tf.random.set_seed(404)
import numpy as np
from PIL import Image
# from google.colab import drive
# drive.mount('/content/drive')

**Default values and file paths**

In [64]:
X_TEST_PATH = '/content/drive/MyDrive/MNIST/digit_xtest.csv'
X_TRAIN_PATH = '/content/drive/MyDrive/MNIST/digit_xtrain.csv'
Y_TRAIN_PATH = '/content/drive/MyDrive/MNIST/digit_ytrain.csv'
Y_TEST_PATH = '/content/drive/MyDrive/MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

In [65]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

**Loading data**

In [66]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 207 ms, sys: 862 µs, total: 208 ms
Wall time: 216 ms


In [67]:
%%time
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 37.1 ms, sys: 0 ns, total: 37.1 ms
Wall time: 39.9 ms


In [68]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 39.3 s, sys: 1.02 s, total: 40.3 s
Wall time: 40.3 s


In [69]:
%%time
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 6.62 s, sys: 163 ms, total: 6.78 s
Wall time: 6.8 s


**Data Exploration**

In [70]:
#all black and white hand written digits
x_train_all.shape
#784 = 28*28*1

(60000, 784)

In [71]:
x_test.shape

(10000, 784)

In [72]:
y_train_all.shape

(60000,)

**Data Pre-processing**

In [73]:
#re-scaling values
x_train_all, x_test = x_train_all / 255.0 , x_test/255.0

In [74]:
#doing the process of indexing - one hot encoding
values = y_train_all[:5]
np.eye(10)[values] #creates sparse matrix #2D array with 1 in diag

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

**Convert values to one hot encoding**

In [75]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [76]:
y_train_all.shape

(60000, 10)

In [77]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

**creating validation data from training data**

In [78]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]


In [79]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

**Setting up tensorflow graph**

In [80]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS])
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES])

**Neural Network Architecture**

**Hyperparameters**

In [120]:
nr_epochs = 50
learning_rate = 1e-3 #0.001
n_hidden1 = 512
n_hidden2 = 64

In [82]:
#initailize weights for layer 1
initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
w1 = tf.Variable(initial_value = initial_w1)

In [83]:
#biases
initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
b1 = tf.Variable(initial_value=initial_b1)

In [84]:
#op => rawinput and weights + bias
layer1_in = tf.matmul(X,w1) + b1


In [85]:
#defining output layer
layer1_out = tf.nn.relu(layer1_in)

In [86]:
#initailize weights for layer2
initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
w2 = tf.Variable(initial_value = initial_w2)

#biases
initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
b2 = tf.Variable(initial_value=initial_b2)

#op => rawinput and weights + bias
layer2_in = tf.matmul(layer1_out,w2) + b2

#defining output layer
layer2_out = tf.nn.softmax(layer2_in)



In [87]:
#defining output layer
#initailize weights for layer2
initial_w3 = tf.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
w3 = tf.Variable(initial_value = initial_w3)

#biases
initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
b3 = tf.Variable(initial_value=initial_b3)

#op => rawinput and weights + bias
layer3_in = tf.matmul(layer2_out,w3) + b3

#defining output layer
output = tf.nn.softmax(layer3_in)


**Loss, optimisation, defining metrics**

In [88]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

In [89]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

**Accuracy metrics**

In [90]:
correct_prediction = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#argmax => index with maximum value

**Initiating sessions**

In [121]:
sess = tf.Session()

In [122]:
init = tf.global_variables_initializer()
sess.run(init)

In [123]:
b3.eval(sess)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

**Data batches**

In [124]:
size_of_batch = 1000
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch =0

In [125]:
def next_batch(batch_size, data, labels): #1000 per batch (0-999) => 1000 to 2000
  global num_examples
  global index_in_epoch

  start = index_in_epoch
  index_in_epoch+=batch_size

  if index_in_epoch > num_examples:
    start=0
    index_in_epoch=batch_size

  end= index_in_epoch

  return data[start:end], labels[start:end]

**Training data**

In [126]:
for epoch in range(nr_epochs):
  for i in range(nr_iterations):
    batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
    # feed dictionary
    feed_dictionary = {X:batch_x, Y:batch_y}
    #run the calc
    sess.run(train_step, feed_dict= feed_dictionary)
    batch_accuracy = sess.run(fetches=[accuracy], feed_dict = feed_dictionary)
  #print(f'Epoch {epoch} \t Training accuracy = {batch_accuracy}')
print('Training done')



Training done


**Making predictions**

In [127]:
img = Image.open("/content/drive/MyDrive/MNIST/test_img.png")
img

In [128]:
##convert and process the image to grayscale
bw = img.convert('L')
img_array = np.invert(bw)

In [129]:
img_array.shape

(28, 28)

In [130]:
test_img = img_array.ravel() # returns a flattened array

In [131]:
test_img.shape

(784,)

In [132]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [133]:
print(f'predicted value is {prediction}')

predicted value is [2]


**Testing and Evaluation**

In [134]:
#accuracy on test data
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'accuracy on test data {test_accuracy:0.2%}')

accuracy on test data 78.47%


In [135]:
sess.close()